In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
import pathlib

dataset_path = "/kaggle/input/food-freshness-dataset/Dataset"
dataset_path = pathlib.Path(dataset_path)  # <-- convert string to Path


In [ ]:
# Filter only valid image extensions
valid_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]
all_image_paths = [str(p) for p in dataset_path.rglob("*") if p.suffix.lower() in valid_extensions]

print(f"Total valid images: {len(all_image_paths)}")

# Check for corrupted images
for img_path in all_image_paths:
    try:
        img = Image.open(img_path)
        img.verify()  # verify that it is not corrupted
    except Exception as e:
        print(f"Corrupted image: {img_path}")


In [ ]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

print("Class names:", train_ds_raw.class_names)  # get classes before mapping



In [ ]:
from tensorflow.keras.layers import Rescaling

normalization_layer = Rescaling(1./255)

train_ds = train_ds_raw.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.apply(tf.data.experimental.ignore_errors())  # skip bad images

val_ds = val_ds_raw.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.apply(tf.data.experimental.ignore_errors())      # skip bad images


In [ ]:
num_classes = len(train_ds_raw.class_names)

model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_acc:.2f}")


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
img_path = "/kaggle/input/fruits/fruits.jpg"


In [ ]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # same size as training
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0   # normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    return img_array

In [ ]:
img = preprocess_image(img_path)


In [ ]:
pred = model.predict(img)
pred_class = np.argmax(pred, axis=1)[0]

In [ ]:
class_names = train_ds_raw.class_names  # ['Fresh', 'Rotten']
predicted_label = class_names[pred_class]


In [ ]:
print(f"Predicted class: {predicted_label}")

plt.imshow(image.load_img(img_path))
plt.title(f"Prediction: {predicted_label}")
plt.axis("off")
plt.show()


In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Folder path
test_folder = "/kaggle/input/fruits/"

# Collect all valid image paths
valid_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]
test_image_paths = [os.path.join(test_folder, f) for f in os.listdir(test_folder)
                    if os.path.splitext(f)[1].lower() in valid_extensions]

# Initialize counters for each class
class_names = train_ds_raw.class_names  # ['Fresh', 'Rotten']
class_counts = {cls: 0 for cls in class_names}

# Predict each image
for img_path in test_image_paths:
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    pred_class = np.argmax(pred, axis=1)[0]
    class_label = class_names[pred_class]
    class_counts[class_label] += 1

    print(f"{os.path.basename(img_path)} ➜ {class_label}")

# Summary
print("\n=== Summary ===")
total_images = len(test_image_paths)
for cls, count in class_counts.items():
    print(f"{cls}: {count}")
print(f"Total images: {total_images}")

# Visualization
plt.bar(class_counts.keys(), class_counts.values(), color=["green", "red"])
plt.title("Fresh vs Rotten Fruit Count")
plt.ylabel("Number of Images")
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img_path = "/kaggle/input/fruits/fruits.jpg"
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis("off")
plt.show()


In [ ]:
# Convert to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

# Define a mask to extract colored areas (avoid background)
mask = cv2.inRange(hsv, (0, 40, 40), (179, 255, 255))
result = cv2.bitwise_and(image, image, mask=mask)

plt.imshow(result)
plt.axis("off")
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
image = cv2.imread("/kaggle/input/fruits/fruits.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert to HSV color space (better for color-based segmentation)
hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

# Define color ranges for typical fruit colors (tune if needed)
color_ranges = {
    "red": [(0, 80, 80), (10, 255, 255)],       # strawberries, apples
    "yellow": [(20, 80, 80), (35, 255, 255)],   # bananas
    "orange": [(10, 100, 100), (25, 255, 255)], # oranges
    "green": [(35, 60, 60), (85, 255, 255)],    # apples, unripe fruits
}

# Create one combined mask for all colors
mask_total = np.zeros(hsv.shape[:2], dtype=np.uint8)
for color, (lower, upper) in color_ranges.items():
    lower = np.array(lower)
    upper = np.array(upper)
    mask = cv2.inRange(hsv, lower, upper)
    mask_total = cv2.bitwise_or(mask_total, mask)

# Morphological operations to separate touching fruits
kernel = np.ones((7, 7), np.uint8)
mask_clean = cv2.morphologyEx(mask_total, cv2.MORPH_OPEN, kernel)
mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel)

# Find contours
contours, _ = cv2.findContours(mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

objects = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 40 and h > 40:  # ignore small noise
        objects.append((x, y, w, h))
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 3)

# Show results
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.title(f"Detected {len(objects)} possible fruits")
plt.show()

print(f"Detected {len(objects)} possible fruits.")
